In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage import median_filter, rotate, shift
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [2]:
df = pd.read_pickle("/home/unsi/다운로드/LSWMD.pkl")  #/home/unsi/다운로드/LSWMD.pkl /Users/unsi/Downloads/LSWMD.pkl
#ㅇㅇ 

In [3]:
class2idx = {
    'Loc': 0,
    'Edge-Loc': 1,
    'Center': 2,
    'Edge-Ring': 3,
    'Scratch': 4,
    'Random': 5,
    'Near-full': 6,
    'Donut': 7,
    'pass': 8
}

In [4]:
tmp = df.loc[0]

In [5]:
tmp['waferMap'].shape

(45, 48)

In [6]:
df = df[df['waferMap'].apply(lambda x: x.shape == (26, 26))]

In [7]:
df

,waferMap,dieSize,lotName,waferIndex,trianTestLabel,failureType
150,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1, 1, 2,...",533.0,lot7,1.0,[[Training]],[[none]]
151,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1,...",533.0,lot7,2.0,[[Training]],[[none]]
152,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 1, 1,...",533.0,lot7,3.0,[[Training]],[[none]]
153,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 1, 1,...",533.0,lot7,4.0,[[Training]],[[none]]
154,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2,...",533.0,lot7,5.0,[[Training]],[[none]]
...,...,...,...,...,...,...
679275,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 1, 1,...",533.0,lot42180,20.0,[],[]
679276,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 2, 2,...",533.0,lot42180,21.0,[],[]
679277,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 1, 1,...",533.0,lot42180,22.0,[],[]
679278,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 1, 2,...",533.0,lot42180,23.0,[],[]


In [8]:
import numpy as np

# class2idx 딕셔너리 (예시)
class2idx = {
    'Loc': 0,
    'Edge-Loc': 1,
    'Center': 2,
    'Edge-Ring': 3,
    'Scratch': 4,
    'Random': 5,
    'Near-full': 6,
    'Donut': 7
}

'''def map_failure_type(x):
    # 'none' 문자열 처리
    if isinstance(x, str) and x == 'none':
        return 8
    # None 값 처리
    elif x is None:
        return 8
    # 배열 처리
    elif isinstance(x, np.ndarray):
        # 배열이 비어있는지 확인
        if x.size == 0:
            return 8
        else:
            # 배열의 모든 값이 'none'인 경우 처리
            first_element = str(x[0])
            return class2idx.get(first_element, 8)
    # 리스트 처리
    elif isinstance(x, list):
        # 리스트가 비어있는지 확인
        if len(x) == 0:
            return 8
        else:
            # 리스트의 첫 번째 값 사용
            first_element = str(x[0])
            return class2idx.get(first_element, 8)
    # 그 외 (정상 값 또는 다른 형태의 값 처리)
    else:
        return class2idx.get(str(x), 8)'''


def map_failure_type(failure_list):
    # numpy array를 리스트로 변환
    if isinstance(failure_list, np.ndarray):
        failure_list = failure_list.tolist()

    # 리스트 안에 리스트가 있는 경우 첫 번째 원소를 가져오기
    while isinstance(failure_list, list) and len(failure_list) > 0 and isinstance(failure_list[0], list):
        failure_list = failure_list[0]

    if not failure_list or failure_list[0] in ['none', None]:
        return 8
    return class2idx.get(failure_list[0], 8)


# apply로 처리
df['failureType'] = df['failureType'].apply(map_failure_type)

# 결과 확인
print(df['failureType'])


150       8
151       8
152       8
153       8
154       8
         ..
679275    8
679276    8
679277    8
679278    8
679279    8
Name: failureType, Length: 30078, dtype: int64


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30078 entries, 150 to 679279
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   waferMap        30078 non-null  object 
 1   dieSize         30078 non-null  float64
 2   lotName         30078 non-null  object 
 3   waferIndex      30078 non-null  float64
 4   trianTestLabel  30078 non-null  object 
 5   failureType     30078 non-null  int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 1.6+ MB


In [10]:
df[df['failureType'] != 8]

,waferMap,dieSize,lotName,waferIndex,trianTestLabel,failureType
186,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 1, 1,...",533.0,lot8,12.0,[[Training]],3
264,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 2, 1,...",533.0,lot11,15.0,[[Training]],1
377,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 2, 2,...",533.0,lot16,3.0,[[Training]],1
381,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2,...",533.0,lot16,7.0,[[Training]],1
397,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 2, 2,...",533.0,lot16,23.0,[[Training]],1
...,...,...,...,...,...,...
679154,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1, 1, 1,...",533.0,lot42175,24.0,[[Test]],0
679165,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1, 1, 1,...",533.0,lot42176,10.0,[[Test]],2
679167,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 1, 2,...",533.0,lot42176,12.0,[[Test]],1
679176,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1, 1, 1,...",533.0,lot42176,21.0,[[Test]],1


In [11]:
feature = df['waferMap']
target = df['failureType']

In [12]:
from scipy.ndimage import rotate


# 1. 필터링 - 미디언 필터 적용
def apply_median_filter_to_features(features, targets):
    filtered_maps = []
    corresponding_targets = []

    # feature와 target을 동시에 순회
    for wafer_map, failure_type in zip(features, targets):
        # 관심 있는 영역(1과 2)에만 필터 적용
        filtered_map = np.where(wafer_map == 0, 0, median_filter(wafer_map, size=3))

        # 필터링된 결과와 target을 각각 리스트에 추가
        filtered_maps.append(filtered_map)
        corresponding_targets.append(failure_type)

    return np.array(filtered_maps), np.array(corresponding_targets)


# 2. 데이터 증강 - 회전 및 뒤집기
def augment_data(wafer_maps, targets):
    augmented_maps = []
    augmented_targets = []

    # feature와 target을 동시에 순회
    for wafer_map, target in zip(wafer_maps, targets):
        # 원본 데이터는 항상 추가
        augmented_maps.append(wafer_map)
        augmented_targets.append(target)

        # 결함이 있는 클래스만 증강 (pass class 8 제외)
        if target != 8:
            # 90도 회전 데이터 추가
            augmented_maps.append(rotate(wafer_map, 90))
            augmented_targets.append(target)

            # 상하 뒤집기 데이터 추가
            augmented_maps.append(np.flip(wafer_map, axis=0))
            augmented_targets.append(target)

            # 좌우 뒤집기 데이터 추가
            augmented_maps.append(np.flip(wafer_map, axis=1))
            augmented_targets.append(target)

    return np.array(augmented_maps), np.array(augmented_targets)



# 3. 특징 추출 - 결함 비율 및 결함 분포
import numpy as np


# 특징 추출 함수
def extract_features(wafer_maps):
    features = []
    for wafer_map in wafer_maps:
        # 전체 die의 개수
        total_die = np.sum(wafer_map > 0)

        # 결함이 있는 die의 개수
        defect_die = np.sum(wafer_map == 2)

        # 결함 비율 계산
        defect_ratio = defect_die / total_die if total_die > 0 else 0

        # 결함이 중앙에 집중되었는지, 가장자리에 집중되었는지 확인
        center_region = wafer_map[9:17, 9:17]  # (26, 26) 해상도에 맞는 중앙 영역
        center_defect_ratio = np.sum(center_region == 2) / defect_die if defect_die > 0 else 0

        # 특징 리스트에 추가 (결함 비율, 중앙 결함 비율)
        features.append([defect_ratio, center_defect_ratio])

    return np.array(features)

In [13]:
filtered_features, filtered_targets = apply_median_filter_to_features(feature, target)
filtered_targets.shape

(30078,)

In [14]:
filtered_targets[filtered_targets != 8]

array([3, 1, 1, 1, 1, 1, 1, 1, 4, 5, 2, 0, 1, 0, 2, 4, 1, 1, 0, 1, 2, 6,
       1, 5, 1, 0, 1, 5, 1, 0, 1, 1, 1, 1, 5, 0, 4, 0, 5, 0, 1, 0, 0, 1,
       1, 1, 5, 0, 0, 6, 5, 5, 1, 1, 1, 4, 1, 1, 2, 1, 0, 0, 1, 1, 4, 1,
       1, 0, 4, 1, 4, 1, 1, 1, 6, 1, 1, 1, 0, 0, 0, 1, 1, 4, 1, 1, 1, 5,
       3, 1, 1, 0, 1, 6, 0, 0, 0, 5, 5, 1, 1, 0, 2, 5, 5, 5, 4, 6, 0, 1,
       5, 4, 1, 5, 1, 0, 1, 5, 5, 1, 6, 0, 2, 4, 0, 1, 3, 4, 0, 0, 4, 4,
       5, 1, 1, 3, 1, 2, 0, 0, 1, 1, 3, 4, 1, 4, 1, 1, 1, 0, 2, 1, 0, 3,
       6, 1, 1, 0, 0, 1, 0, 1, 0, 4, 2, 2, 1, 1, 3, 3, 1, 1, 1, 6, 0, 0,
       1, 0, 2, 1, 3, 4, 5, 1, 4, 1, 0, 3, 4, 3, 1, 2, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 4, 3, 3, 3, 3, 3, 3, 3, 3, 1, 2, 2, 5, 5, 5, 1, 0, 0, 1,
       1, 5, 5, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 5,
       5, 0, 0, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 5, 5, 5,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 2, 0, 0, 1, 1, 0,

In [15]:
print('mean : ', filtered_features.mean())
print('std : ', filtered_features.std())
print('min : ', filtered_features.min())
print('max : ', filtered_features.max())

mean :  0.8029313134961527
std :  0.4337232638426162
min :  0
max :  2


In [16]:
extract_features(filtered_features)

array([[0.01313321, 0.        ],
       [0.00750469, 0.        ],
       [0.00562852, 0.66666667],
       ...,
       [0.00938086, 0.4       ],
       [0.06003752, 0.03125   ],
       [0.00562852, 0.        ]])

In [17]:
filtered_targets.shape


(30078,)

In [18]:
mask = filtered_targets != 7
filtered_features = filtered_features[mask]
filtered_targets = filtered_targets[mask]

In [19]:
from sklearn.model_selection import train_test_split

filtered_targets = np.where(filtered_targets == 8, 7, filtered_targets)
X_train, X_test, Y_train, Y_test = train_test_split(filtered_features, filtered_targets, shuffle=True, random_state=123,
                                                    stratify=filtered_targets)
print(X_train.shape[0])

22557


In [21]:
X_train, Y_train = augment_data(X_train, Y_train)
print(X_train.shape[0])

360912


In [22]:
print(Y_train[Y_train != 8].shape)

(360912,)


In [23]:
X_train = X_train.reshape(-1, 26 * 26)
X_test = X_test.reshape(-1, 26 * 26)

In [24]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from imblearn.over_sampling import ADASYN
from sklearn.metrics import classification_report, accuracy_score

adasyn = ADASYN(random_state=123)
X_train, Y_train = adasyn.fit_resample(X_train, Y_train)

KeyboardInterrupt: 

In [25]:
params_gpu = {
    'objective': 'binary:logistic',
    'tree_method': 'hist',  # Use histogram method
    'device': 'cuda',       # Set device to CUDA for GPU usage
    'eval_metric': 'logloss'
}

model = XGBClassifier(
    objective='multi:softmax',  # 다중 클래스 분류
    eval_metric='mlogloss',  # 다중 클래스 로지스틱 손실
    random_state=123,
    learning_rate=0.1,
    max_depth=6,
    n_estimators=100,
    device = "cuda"
)
model.fit(X_train, Y_train, verbose=True)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cuda', early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, objective='multi:softmax', ...)

In [27]:
from sklearn.metrics import precision_score, recall_score, f1_score

# 모델 예측
y_pred = model.predict(X_test)

# 정확도 계산
accuracy = accuracy_score(Y_test, y_pred)

# precision_score에서 zero_division 매개변수 사용
precision = precision_score(Y_test, y_pred, average='weighted', zero_division=1)
recall = recall_score(Y_test, y_pred, average='weighted')
f1 = f1_score(Y_test, y_pred, average='weighted')

# 결과 출력
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

Accuracy: 0.9762
Precision: 0.9711
Recall: 0.9762
F1 Score: 0.9718


In [28]:
accuracy = accuracy_score(Y_test, y_pred)

# precision_score에서 zero_division 매개변수 사용
precision = precision_score(Y_test, y_pred, average='macro', zero_division=1)
recall = recall_score(Y_test, y_pred, average='macro')
f1 = f1_score(Y_test, y_pred, average='macro')

# 결과 출력
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

Accuracy: 0.9762
Precision: 0.7143
Recall: 0.5078
F1 Score: 0.5482


In [29]:
np.unique(Y_train)

array([0, 1, 2, 3, 4, 5, 6, 7])

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score

model = LogisticRegression(random_state=123, max_iter=500)
model.fit(X_train, Y_train)
y_pred = model.predict(X_test)

KeyboardInterrupt: 

In [29]:
from sklearn.metrics import precision_score, recall_score, f1_score

accuracy = accuracy_score(Y_test, y_pred)
precision = precision_score(Y_test, y_pred, average='weighted', zero_division=1)
recall = recall_score(Y_test, y_pred, average='weighted')
f1 = f1_score(Y_test, y_pred, average='weighted')#macro, weighted

# 결과 출력
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

Accuracy: 0.9672
Precision: 0.9754
Recall: 0.9672
F1 Score: 0.9568


In [30]:
accuracy = accuracy_score(Y_test, y_pred)
precision = precision_score(Y_test, y_pred, average='macro', zero_division=1)
recall = recall_score(Y_test, y_pred, average='macro')
f1 = f1_score(Y_test, y_pred, average='macro')#macro, weighted

# 결과 출력
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

Accuracy: 0.9672
Precision: 0.8795
Recall: 0.2651
F1 Score: 0.1652
